In [137]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset

from tqdm.notebook import tqdm
from tqdm.auto import tqdm

from transformers import BertForSequenceClassification
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [138]:
df_train = pd.read_csv("data/train_English.csv", encoding="ISO-8859-1")
df_test = pd.read_csv("data/test_English.csv", encoding="ISO-8859-1")

In [139]:
df_train['OriginalTweet'][0:1]
df_train[0:0]
df_train[0:1]

,textID,OriginalTweet,selected_text,Sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60


In [140]:
df_train.info()
#location a onek non-null value
#baki kothao non-null nai

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   OriginalTweet     27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   Sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


In [141]:
#drop duplicate tweet
df_train.drop_duplicates(subset='OriginalTweet',inplace=True)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   OriginalTweet     27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   Sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


In [142]:
df_train_temp = df_train[['OriginalTweet','Sentiment']]
df_train.head()['OriginalTweet'].values
df_train = df_train.copy()
df_test_temp = df_train[['OriginalTweet','Sentiment']]
df_test.head()['OriginalTweet'].values
df_test = df_test.copy()

#@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
#mention gula meaning less
#telegram link ero kono kaj nai 
#eigula remove kora lagbe

In [143]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#print(device)
#if device.type == 'cuda':
#    print(torch.cuda.get_device_name(0))
#    print('Memory Usage:')
#    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
#    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
#else:
#    exit(0)
#torch.cuda.empty_cache()

In [144]:
import re
def preprocess(row):
    # https://radimrehurek.com/gensim/parsing/preprocessing.html
    #mainly to remove whitespaces
    #remove all kinds of links from data
    # also remove mentions like @
    #first row should be and and
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', ' ', row)
    # Remove mentions
    text = re.sub(r'@\S+', ' ', text)
    # Remove hashtags
    text = re.sub(r'#', ' ', text)
    text = text.replace('.', ' ').replace(',', ' ')
    # Remove newline and carriage return characters
    text = text.replace('\r', ' ').replace('\n', ' ')
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [145]:
# Replace NaNs with empty strings
df_train['OriginalTweet'] = df_train['OriginalTweet'].fillna('')
df_test['OriginalTweet'] = df_test['OriginalTweet'].fillna('')

# Ensure all entries are strings
df_train['OriginalTweet'] = df_train['OriginalTweet'].astype(str)
df_test['OriginalTweet'] = df_test['OriginalTweet'].astype(str)

In [146]:
df_train['CleanTweet']=df_train['OriginalTweet'].apply(preprocess)
df_train.head()['CleanTweet'].values
df_train = df_train[['CleanTweet','Sentiment']]
df_train['CleanTweet']

df_test['CleanTweet']=df_test['OriginalTweet'].apply(preprocess)
df_test.head()['CleanTweet'].values
df_test = df_test[['CleanTweet','Sentiment']]
df_test['CleanTweet']

0                                 Last session of the day
1       Shanghai is also really exciting precisely sky...
2       Recession hit Veronique Branquinho she has to ...
3                                              happy bday
4                                               I like it
                              ...                        
3529    its at 3 am im very tired but i cant sleep but...
3530    All alone in this old house again Thanks for t...
3531    I know what you mean My little dog is sinking ...
3532    _sutra what is your next youtube video gonna b...
3533                               omgssh ang cute ng bby
Name: CleanTweet, Length: 3534, dtype: object

In [147]:
df_train['CleanTweet'][1:8].values

array(['Sooo SAD I will miss you here in San Diego',
       'my boss is bullying me', 'what interview leave me alone',
       'Sons of why couldnt they put them on the releases we already bought',
       'some shameless plugging for the best Rangers forum on earth',
       '2am feedings for the baby are fun when he is all smiles and coos',
       'Soooo high'], dtype=object)

In [148]:
df_train=df_train[df_train['CleanTweet'] != ' ']
df_test=df_test[df_test['CleanTweet'] != ' ']
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CleanTweet  27481 non-null  object
 1   Sentiment   27481 non-null  object
dtypes: object(2)
memory usage: 429.5+ KB


In [149]:
#now we need to work with Sentiment values...they are 5 types 
print(df_train['Sentiment'].value_counts())
df_train['Sentiment'] = df_train['Sentiment'].map({'negative':0,'neutral':1,'positive':2})
print(df_train['Sentiment'].value_counts())

print(df_test['Sentiment'].value_counts())
df_test['Sentiment'] = df_test['Sentiment'].map({'negative':0,'neutral':1,'positive':2})
print(df_test['Sentiment'].value_counts())

Sentiment
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64
Sentiment
1    11118
2     8582
0     7781
Name: count, dtype: int64
Sentiment
neutral     1430
positive    1103
negative    1001
Name: count, dtype: int64
Sentiment
1    1430
2    1103
0    1001
Name: count, dtype: int64


In [150]:
#print(df_train['sentiment'].isna().sum())

In [151]:
#df_train = df_train.dropna(subset=['sentiment'])

In [152]:
X_train, X_val, y_train, y_val = train_test_split(df_train.index.values, 
                                                  df_train.Sentiment.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df_train.Sentiment.values)
print(X_train)
df_train['data_type'] = ['not_set']*df_train.shape[0]
#mark training and validation data
df_train.loc[X_train, 'data_type'] = 'train'
df_train.loc[X_val, 'data_type'] = 'val'
df_train[:]
df_train['data_type'].value_counts()

[14668 20766 23037 ... 11017 25564 18322]


data_type
train    23358
val       4123
Name: count, dtype: int64

In [153]:
#Initiating BertTokenizer from 'bert-base-uncased' model.
from transformers import AutoModelForSequenceClassification, AutoTokenizer 
model_name = "google/muril-base-cased" 
from transformers import BertTokenizer, ElectraForSequenceClassification, RobertaForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [154]:
df_train[df_train.data_type=='train'].CleanTweet.values

array(['Id have responded if I were going',
       'Sooo SAD I will miss you here in San Diego',
       'my boss is bullying me', ...,
       'Yay good for both of you Enjoy the break you probably need it after such hectic weekend Take care hun xxxx',
       'But it was worth it',
       'All this flirting going on The ATG smiles Yay hugs'], dtype=object)

In [155]:
encoded_data_train = tokenizer.batch_encode_plus(
    df_train[df_train.data_type=='train'].CleanTweet.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest',
    truncation=True, 
    return_tensors='pt'
)

In [156]:
encoded_data_test = tokenizer.batch_encode_plus(
    df_test.CleanTweet.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest',
    truncation=True, 
    return_tensors='pt'
)

In [157]:
# input_ids_train = encoded_data_train['input_ids']
# This line extracts the input_ids from the encoded_data_train dictionary and assigns them to the variable input_ids_train.
# encoded_data_train likely contains the tokenized input data, where input_ids are numerical representations of the tokens.


# attention_masks_train = encoded_data_train['attention_mask']
# This line extracts the attention_mask from the encoded_data_train dictionary and assigns them to the variable attention_masks_train.
# attention_masks are used in models like BERT to differentiate between actual tokens and padding tokens, ensuring that the model only attends to the real tokens.



# labels_train = torch.tensor(df_train[df_train.data_type=='train'].Sentiment.values)
# This line creates a tensor of training labels using the torch.tensor function from the PyTorch library.
# df_train is likely a DataFrame containing the training data.
# df_train[df_train.data_type=='train'] filters the DataFrame to include only the rows where the data_type column is equal to 'train'.
# .Sentiment.values extracts the sentiment labels from the filtered DataFrame.
# The resulting labels are converted into a PyTorch tensor, which can be used in the training process.



input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_train[df_train.data_type=='train'].Sentiment.values)

In [158]:
print(input_ids_train)
print(attention_masks_train)
print(labels_train)

tensor([[  101,  8909,  2031,  ...,     0,     0,     0],
        [  101, 17111,  2080,  ...,     0,     0,     0],
        [  101,  2026,  5795,  ...,     0,     0,     0],
        ...,
        [  101,  8038,  2100,  ...,     0,     0,     0],
        [  101,  2021,  2009,  ...,     0,     0,     0],
        [  101,  2035,  2023,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([1, 0, 0,  ..., 2, 2, 1])


In [159]:
encoded_data_val = tokenizer.batch_encode_plus(
    df_train[df_train.data_type=='val'].CleanTweet.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='longest',
    truncation=True, 
    return_tensors='pt'
)
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df_train[df_train.data_type=='val'].Sentiment.values)

In [160]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(df_test.Sentiment.values)

In [161]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)
print(len(dataset_train), len(dataset_val))
print(len(dataset_test))
#training set ar test set er encoding thik ase // total count o ok

23358 4123
3534


In [162]:
model = RobertaForSequenceClassification.from_pretrained("bert-base-uncased",
                                                          num_labels=3,
                                                          output_attentions=False,
                                                          output_hidden_states=False)
# num_labels==  specifies the number of output labels the model should predict.
# output_attentions=False and output_hidden_states=False are optional parameters that,
# when set to False, prevent the model from returning attention weights and hidden states, respectively, which can save memory during training and inference

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.

In [163]:
batch_size = 4
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)
dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)
dataloader_test = DataLoader(dataset_test,
                             sampler=SequentialSampler(dataset_test),
                             batch_size=batch_size)

In [164]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [165]:
#we fix epoch number and scheduler
epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [166]:
import torch

def save_checkpoint(epoch, model, optimizer, scheduler, file_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict()
    }, file_path)

In [167]:
for epoch in tqdm(range(1, epochs + 1)):
    # Training code...

    # Save the checkpoint
    save_checkpoint(epoch, model, optimizer, scheduler, 'checkpoint.pth')

  0%|          | 0/10 [00:00<?, ?it/s]

In [168]:
def load_checkpoint(file_path, model, optimizer, scheduler):
    checkpoint = torch.load(file_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch']
    return start_epoch

In [169]:
# Assuming you have the model, optimizer, and scheduler already defined
start_epoch = 0
checkpoint_path = 'checkpoint.pth'

try:
    start_epoch = load_checkpoint(checkpoint_path, model, optimizer, scheduler)
    print(f"Resuming from epoch {start_epoch + 1}")
except FileNotFoundError:
    print("No checkpoint found, starting from scratch.")

for epoch in tqdm(range(start_epoch + 1, epochs + 1)):
    model.train()

    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc=f'Epoch {epoch}', leave=False, disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)

        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2],
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': f'{loss.item() / len(batch):.3f}'})

    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

    # Save the checkpoint
    save_checkpoint(epoch, model, optimizer, scheduler, checkpoint_path)

Resuming from epoch 11


0it [00:00, ?it/s]

In [170]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [171]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [172]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [173]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [174]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
                 
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/5840 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.9285103312986966
Validation loss: 0.8339783930292879
F1 Score (Weighted): 0.6225144668162947


Epoch 2:   0%|          | 0/5840 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.7546149021601432
Validation loss: 0.84985072955229
F1 Score (Weighted): 0.7066631227351663


...


Epoch 8
Training loss: 0.6187493674791169
Validation loss: 1.2705538887729295
F1 Score (Weighted): 0.7225381033258682


Epoch 9:   0%|          | 0/5840 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.5780780265859068
Validation loss: 1.3857478378706447
F1 Score (Weighted): 0.7150732165131427


Epoch 10:   0%|          | 0/5840 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.5561710679701694
Validation loss: 1.378443002393564
F1 Score (Weighted): 0.7175830190775107


In [175]:
def result_generation(preds, labels):
    label_dict = {'negative':0,'neutral':1,'positive':2}
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

        P_c = precision_score(labels_flat, preds_flat, average=None, labels=[label])[0]
        R_c = recall_score(labels_flat, preds_flat, average=None, labels=[label])[0]
        F1_c = f1_score(labels_flat, preds_flat, average=None, labels=[label])[0]

        print(f"=*= {label_dict_inverse[label]} =*=")
        # print("Full precision:\t",P_c)
        # print("Full recall:\t\t",R_c)
        # print("Full F1 score:\t",F1_c)
        print(f"precision:\t{P_c:.4f}")
        print(f"recall:\t\t{R_c:.4f}")
        print(f"F1 score:\t{F1_c:.4f}")
        print()

    P = precision_score(labels_flat, preds_flat, average='micro')
    R = recall_score(labels_flat, preds_flat, average='micro')
    F1 = f1_score(labels_flat, preds_flat, average='micro')

    print("=*= global =*=")7
    print(f"precision:\t{P:.4f}")
    print(f"recall:\t\t{R:.4f}")
    print(f"F1 score:\t{F1:.4f}")
    print()

In [176]:
def evaluateTest(model, dataloader_val):
    
    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                  }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [177]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [178]:
_, predictions, true_vals = evaluateTest(model, dataloader_test)
result_generation(predictions, true_vals)

Class: negative
Accuracy: 722/1001

=*= negative =*=
precision:	0.7010
recall:		0.7213
F1 score:	0.7110

Class: neutral
Accuracy: 998/1430

=*= neutral =*=
precision:	0.6911
recall:		0.6979
F1 score:	0.6945

Class: positive
Accuracy: 850/1103

=*= positive =*=
precision:	0.8019
recall:		0.7706
F1 score:	0.7859

=*= global =*=
precision:	0.7272
recall:		0.7272
F1 score:	0.7272

